# Ping data exploration

## How data is  collected
Ping data is collected every 5 seconds doing single pings to 8.8.8.8

**Example**:
>$ ping -c 1 8.8.8.8  
>PING 8.8.8.8 (8.8.8.8): 56 data bytes  
>64 bytes from 8.8.8.8: icmp_seq=0 ttl=119 **time=22.985 ms**  
>--- 8.8.8.8 ping statistics ---  
>1 packets transmitted, 1 packets received, **0.0% packet loss**  
>round-trip min/**avg**/max/**stddev** = 22.985/22.985/**22.985**/**0.000** ms  

Data being collected:
 - **Ping latency** (in miliseconds, 22.985 ms in the example)
 - __Percentage of packets dropped__  (0.0% in the example, can be only 0% or 100%, data collected for single pings) 
 - **Standart deviation (for latency)**  - always 0 for single pings


## How data looks like
Load libraries:

In [1]:
from data_exploration import *

Set up test time interval:

In [2]:
time_interval='4w' #5d

Set up influxdb connection:

In [3]:
client, client_df = connect_to_influxdb()

Checking last 10 records in the ping measurment and see how data looks like:

In [4]:
query_ping = 'SELECT PING,SK_PI FROM PING ORDER BY time DESC LIMIT 10;'
ping_df = get_dataframe_from_influxdb(client_df=client_df,query_influx=query_ping,table_name='PING')
ping_df

,time,PING,SK_PI
6,2019-01-18 23:59:57,14.757,3
2,2019-01-18 23:59:54,37.799,7
7,2019-01-18 23:59:58,29.995,7
3,2019-01-18 23:59:54,32.705,9
5,2019-01-18 23:59:55,27.534,11
4,2019-01-18 23:59:55,26.040,12
9,2019-01-19 00:00:00,48.096,12
1,2019-01-18 23:59:54,35.011,15
0,2019-01-18 23:59:52,816.015,16
8,2019-01-18 23:59:58,797.279,16


Let's take just one device, for example 3:

In [5]:
query_ping = "SELECT PING,SK_PI FROM PING WHERE SK_PI='3' ORDER BY time  DESC LIMIT 10 ;"
ping_df = get_dataframe_from_influxdb(client_df=client_df,query_influx=query_ping,table_name='PING')
ping_df

,time,PING,SK_PI
0,2019-01-18 23:59:11,14.612,3
1,2019-01-18 23:59:17,14.638,3
2,2019-01-18 23:59:21,14.760,3
3,2019-01-18 23:59:27,14.755,3
4,2019-01-18 23:59:31,14.631,3
5,2019-01-18 23:59:37,14.712,3
6,2019-01-18 23:59:41,14.728,3
7,2019-01-18 23:59:47,14.766,3
8,2019-01-18 23:59:51,14.623,3
9,2019-01-18 23:59:57,14.757,3


Let's compare with what we have in MS SQL database:

In [6]:
cnxn = connect_to_mssql()
sql = "SELECT TOP 10 DATA_DATE, PING, SK_PI,CONNTRACK FROM FCT_PI WHERE SK_PI='3' ORDER BY DATA_DATE DESC;"
pd.read_sql(sql,cnxn)

,DATA_DATE,PING,SK_PI,CONNTRACK
0,2019-01-19 23:59:59.400,NaN,3,15
1,2019-01-19 23:59:57.397,14.669,3,15
2,2019-01-19 23:59:55.397,NaN,3,15
3,2019-01-19 23:59:53.393,14.765,3,15
4,2019-01-19 23:59:51.390,NaN,3,15
5,2019-01-19 23:59:49.390,NaN,3,16
6,2019-01-19 23:59:47.387,14.633,3,16
7,2019-01-19 23:59:45.387,NaN,3,16
8,2019-01-19 23:59:43.383,14.657,3,17
9,2019-01-19 23:59:41.380,NaN,3,19


Pings are coming every 4-6 seconds.

Checking statistics for number of packets dropped:

In [7]:
query_ping = "SELECT DISTINCT(PING_DROPRATE) from PING;"
ping_df = get_dataframe_from_influxdb(client_df=client_df,query_influx=query_ping,table_name='PING')
ping_df

,time,distinct
0,1970-01-01,0
1,1970-01-01,100


Lets see what is the value of *PING* measurment when *PING_DROPRATE* is equal to 100:

In [8]:
query_ping = "SELECT PING, PING_DROPRATE FROM PING WHERE PING_DROPRATE=100 ORDER BY time DESC LIMIT 5;"
ping_df = get_dataframe_from_influxdb(client_df=client_df,query_influx=query_ping,table_name='PING')
ping_df

,time,PING,PING_DROPRATE
0,2019-01-18 23:57:55,0,100
1,2019-01-18 23:57:58,0,100
2,2019-01-18 23:58:12,0,100
3,2019-01-18 23:59:12,0,100
4,2019-01-18 23:59:42,0,100


*PING* is equal to 0 when PING_DROPRATE=100, its importatnt to note.

## Number of datapoints per device

Getting device numbers(tags SK_PI):

In [9]:
device_numbers=get_tag_values_influxdb(client_influx=client,table_name='PING', tag_name='SK_PI')
device_numbers=list(map(int, device_numbers))
device_numbers= sorted(device_numbers)
print(device_numbers)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]


Getting number of data points per device for the entire period of time.

In [10]:
query_ping_counts = 'SELECT COUNT(PING) FROM PING GROUP BY SK_PI;'
ping_counts=get_stats_influxdb(client_influx=client,
                               query_influx=query_ping_counts,
                               stat_name='count',
                               device_numbers=device_numbers)

Plotting device numbers and number of data points. 

In [11]:
simple_bar_plot(xvalues=device_numbers,
                yvalues=ping_counts,
                name="ping datapoints",
                title="Number of data points per device",
                ytitle="Number of datapoints")

Some of the devices have small number of datapoints, may be they are just installed? Lets check how many dataponts came in last 4 weeks.

Getting number of datapoints per device in last 4 weeks and list of the devices that have data for this period of time(not all of the have).

In [12]:
#query_ping_counts_dec = "SELECT COUNT(PING) FROM PING WHERE time >= '2018-12-01 00:00:00' GROUP BY SK_PI ;"
query_ping_counts_time = "SELECT COUNT(PING) FROM PING WHERE time >= now()-"+time_interval+" GROUP BY SK_PI ;"
ping_counts_time = get_stats_influxdb(client_influx=client,
                                      query_influx=query_ping_counts_time,
                                      stat_name='count',
                                      device_numbers=device_numbers)

Plotting combined barchart - entire number of datapoints vs number of datapoints in last 4 weeks.

In [13]:
combined_bar_plot_2traces(xvalues=device_numbers,
                          yvalues1=ping_counts_time,
                          yvalues2=[a - b for a, b in zip(ping_counts, ping_counts_time)],
                          name1='Last '+time_interval,
                          name2='The rest of the time',
                          title="Comparing number of datapoints in last "+time_interval+" vs entire time",
                          ytitle="Number of datapoints")

Devices 1,2,4,5,6,8 and possibly 13 and 14 need to be double checked. Looks like they have started reporting and then stopped. Let's check last reporting time for every device.

In [14]:
query_ping_last = "SELECT LAST(PING), time FROM PING GROUP BY SK_PI;"
result_ping_last=get_stats_influxdb(client_influx=client,
                               query_influx=query_ping_last,
                               stat_name='time',
                               device_numbers=device_numbers)

In [15]:
query_ping_first = "SELECT FIRST(PING), time FROM PING GROUP BY SK_PI;"
result_ping_first=get_stats_influxdb(client_influx=client,
                               query_influx=query_ping_first,
                               stat_name='time',
                               device_numbers=device_numbers)

In [16]:
print("Collectd reporting times:")
data=[]
for i in range(len(device_numbers)):
    try:
        result_ping_first[i] = dateutil.parser.parse(result_ping_first[i]).strftime('%Y-%m-%d %H:%M:%S')
    except:
        result_ping_first[i]=None
    try:
        result_ping_last[i] = dateutil.parser.parse(result_ping_last[i]).strftime('%Y-%m-%d %H:%M:%S')
    except:
        result_ping_last[i]=None
    print("Device: ", device_numbers[i],"  was reporting from ", result_ping_first[i], " to ",result_ping_last[i])
    trace = go.Scatter(x=[result_ping_first[i],result_ping_last[i]],y=[device_numbers[i],device_numbers[i]], 
                       name = device_numbers[i],marker=dict(color=colors[i]))
    data.append(trace)
layout = dict(title = "Device reporting times(collectd)",xaxis=dict(title="Time"),
        yaxis=dict(title="Device Number"))
fig = go.Figure(data=data, layout=layout)
iplot(fig)

Collectd reporting times:
Device:  1   was reporting from  2018-10-10 14:16:15  to  2018-10-11 15:46:31
Device:  2   was reporting from  2018-10-11 15:27:46  to  2018-11-06 19:16:32
Device:  3   was reporting from  2018-10-11 15:27:46  to  2019-01-18 23:59:57
Device:  4   was reporting from  2018-10-11 15:27:46  to  2018-12-05 22:53:02
Device:  5   was reporting from  2018-10-11 15:27:46  to  2018-11-05 21:32:30
Device:  6   was reporting from  2018-10-11 15:27:46  to  2018-10-31 14:55:00
Device:  7   was reporting from  2018-11-11 00:00:03  to  2019-01-18 23:59:58
Device:  8   was reporting from  2018-10-11 15:27:46  to  2019-01-14 14:27:00
Device:  9   was reporting from  2018-10-11 15:27:46  to  2019-01-18 23:59:54
Device:  10   was reporting from  2018-10-11 15:27:46  to  2018-12-31 04:39:59
Device:  11   was reporting from  2018-10-11 15:27:46  to  2019-01-18 23:59:55
Device:  12   was reporting from  2018-10-11 15:27:46  to  2019-01-19 00:00:00
Device:  13   was reporting from  2

Something is happening with devices  1,2,4,5,6,8,13 and 14. They stopped reporting - does it need to be investigated?

## Ping latency 

What is normal ping latency? From [this link](https://www.pingman.com/kb/article/what-s-normal-for-latency-and-packet-loss-42.html):

>There are two normal factors that significantly influence the latency of a consumer device (like a cable modem, dsl modem or dial-up modem).

>The latency of the connecting device. For a cable modem, this can normally be between 5 and 40 ms. For a DSL modem this is normally 10 to 70ms. For a dial-up modem, this is normally anywhere from 100 to 220ms. For a cellular link, this can be from 200 to 600 ms. For a T1, this is normally 0 to 10 ms.
The distance the data is traveling. Data travels at (very roughly) 120,000 miles (or 192,000 kilometers) per second, or 120 miles (192 km) per ms (millisecond) over a network connection. With traceroute, we have to send the data there and back again, so roughly 1 ms of latency is added for every 60 miles (96km, although with the level of accuracy we're using here, we should say '100km') of distance between you and the target.
Connecting to a web site across 1500 miles (2400 km) of distance is going to add at least 25 ms to the latency. Normally, it's more like 75 after the data zig-zags around a bit and goes through numerous routers.

>This means that a DSL modem on the west coast of the United States, tracing to a server on the east coast of the United States should expect somewhere around 120 ms (depending on the route and a number of other factors, but this is a rough ballpark) - 25 ms for the DSL modem and 100 ms for the distance. Tracing across an ocean, or through a satellite link, or some other link where the distance is further will certainly impact the expected latency more.



Let's check actual ping latency numbers in last 4 weeks:

In [17]:
#query_ping_max_dec = "SELECT MAX(PING), MEAN(PING) FROM PING WHERE time >= '2018-12-01 00:00:00' GROUP BY SK_PI;"
#time_interval='4w'
query_ping_stats_time = "SELECT MAX(PING), MEAN(PING), MEDIAN(PING) FROM PING WHERE PING!=0 AND time >= now()-"+\
                         time_interval+" GROUP BY SK_PI;"
result_ping_stats=get_3_stats_influxdb(client_influx=client,
                                       query_influx=query_ping_stats_time,
                                       stat_name1='max',
                                       stat_name2='mean',
                                       stat_name3='median',
                                       device_numbers=device_numbers)

Plotting mean and max ping latency results for every reporting result in December.

In [18]:
combined_bar_plot_3traces(xvalues=result_ping_stats["SK_PI"],
                         yvalues1=result_ping_stats["max"],
                         yvalues2=result_ping_stats["mean"],
                         yvalues3=result_ping_stats["median"],
                         name1="Max",
                         name2="Mean",
                         name3="Median",
                         title="Maximum, mean and median ping delay in miliseconds for the last "+time_interval,
                         stack=False)

Device 16 has the highest mean and highest max.Devices 7,10,12,17 and 18 have large ping delays as well.

### Ping latency grouped by duration
Lets divide ping latencies into 3 groups:
 - group1:  1-40Ms delay
 - group2:  41-100Ms delay
 - group3:  101+Ms delay  
 And calculate percentages of ping latencies in each group for every device:

In [19]:
range1=40
range2=100
#time_interval='4w'

In [20]:
query_ping_group1 = "SELECT COUNT(PING) FROM PING WHERE PING > 0 AND PING < "+str(range1+1)+\
                   " AND time >= now()-"+time_interval+" GROUP BY SK_PI;"
ping_group1_time  = get_stats_influxdb(client_influx=client,
                                      query_influx=query_ping_group1,
                                      stat_name='count',
                                      device_numbers=device_numbers)

In [21]:
query_ping_group2 = "SELECT COUNT(PING) FROM PING WHERE PING > "+str(range1)+\
                   " AND PING < "+str(range2+1)+" AND time >= now()-"+time_interval+" GROUP BY SK_PI;"
ping_group2_time = get_stats_influxdb(client_influx=client,
                                      query_influx=query_ping_group2,
                                      stat_name='count',
                                      device_numbers=device_numbers)

In [22]:
query_ping_group3= "SELECT COUNT(PING) FROM PING WHERE PING > "+str(range2+1)+" AND  time >= now()-"+\
                   time_interval+" GROUP BY SK_PI;"
ping_group3_time = get_stats_influxdb(client_influx=client,
                                      query_influx=query_ping_group3,
                                      stat_name='count',
                                      device_numbers=device_numbers)

In [23]:
combined_bar_plot_3traces(xvalues=device_numbers,
                         yvalues1=ping_group1_time,
                         yvalues2=ping_group2_time,
                         yvalues3=ping_group3_time,
                         name1="Number of pings with 1-"+str(range1)+"Ms delay",
                         name2="Number of pings with "+str(range1+1)+"-"+str(range2)+"Ms delay",
                         name3="Number of pings with "+str(range2+1)+"+Ms delay",
                         title="Ping latency grouped by duration in the last"+time_interval,
                         ytitle="Number of datapoints")

## Ping droprate 
We will calculate the etite number of packets dropped by device using formula:SUM(PING_DROPRATE)/100

In [24]:
query_pingdroprate_mean_time= "SELECT MEAN(PING_DROPRATE) FROM PING WHERE  time >= now()-"+time_interval+"  GROUP BY SK_PI;"
pingdroprate_mean_time=get_stats_influxdb(client_influx=client,
                               query_influx=query_pingdroprate_mean_time,
                               stat_name='mean',
                               device_numbers=device_numbers)

In [25]:
simple_bar_plot(xvalues=device_numbers,
                yvalues=pingdroprate_mean_time,
                name="Mean",
                title="Average percentage of ping droprate per device in the last "+time_interval,
                ytitle="Percent")

Device 10 has the highest mean - highst percentage of packets dropped - more than 6%.

In [26]:
query_pingdroprate_counts_time = "SELECT COUNT(PING_DROPRATE) FROM PING WHERE time >= now()-"+time_interval+\
                                 " AND PING_DROPRATE>0 GROUP BY SK_PI ;"
pingdroprate_counts_time=get_stats_influxdb(client_influx=client,
                               query_influx=query_pingdroprate_counts_time,
                               stat_name='count',
                               device_numbers=device_numbers)

In [27]:
query_ping_counts_time = "SELECT COUNT(PING) FROM PING WHERE time >= now()-"+time_interval+" GROUP BY SK_PI ;"
ping_counts_time = get_stats_influxdb(client_influx=client,
                                      query_influx=query_ping_counts_time,
                                      stat_name='count',
                                      device_numbers=device_numbers)

In [28]:
print(pingdroprate_counts_time)
print([a - b for a, b in zip(ping_counts_time, pingdroprate_counts_time)])

[0, 0, 12, 0, 0, 0, 1460, 2, 3796, 6531, 466, 4527, 0, 0, 24, 10179, 1017, 1143]
[0, 0, 446978, 0, 0, 0, 445511, 65295, 443194, 115505, 446524, 442464, 0, 0, 446966, 436811, 247673, 247546]


In [29]:
combined_bar_plot_2traces(xvalues=device_numbers,
                          yvalues1=pingdroprate_counts_time,
                          yvalues2=[a - b for a, b in zip(ping_counts_time, pingdroprate_counts_time)],
                          name1='Number of pings dropped',
                          name2='Number of pings delievered',
                          title="Ping droprate(actual number) in the last "+time_interval,
                          ytitle="Number of datapoints")

Device #10 and device #16  have the largest number of packets dropped.

In [30]:
#trace = go.Pie(labels=device_numbers, values=device_dropped_counts)
#data=[trace]
#layout = go.Layout(
#        title="Total number of packets dropped by device"
#    )
#fig = go.Figure(data=data, layout=layout)
#iplot(fig)